In [1]:
# TODO DELETE ME
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# TODO

# find accuracy of Base model (using Spacy Root Verb Dependency)
# find accuracy using vanilla CRF Suite
# explore features and hyperparameters of CRF Suite Algo
# find accuracy using tweaked CRF Suite
# display results using Displacy
# interpret results using eli5

<IPython.core.display.Javascript object>

In [3]:
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd
import math

# to set random seed
import numpy as np

# used to create CRF model
from sklearn_crfsuite import CRF

# used to evaluate model
from sklearn_crfsuite import metrics

# for hyperparameter tuning parameter range
import scipy

# for specifying f1 metrics
from sklearn.metrics import make_scorer

# for cross validation of hyperparameters
from sklearn.model_selection import RandomizedSearchCV

# to visualize the weight of parameters of the fitted model
import eli5

<IPython.core.display.Javascript object>

In [5]:
np.random.seed(42)

<IPython.core.display.Javascript object>

In [6]:
data = pd.read_csv("tagged_data.csv", index_col=0)

<IPython.core.display.Javascript object>

In [7]:
data.head(10)

sentence#     word   pos        tag
0        0.0  Preheat    VB   U-Action
1        0.0     oven    NN  U-Utensil
2        0.0       to   ADP          O
3        0.0      425   NUM          O
4        0.0  degrees  NOUN          O
5        0.0       F.  NOUN          O
6        1.0    Press    NN   U-Action
7        1.0    dough  NOUN          O
8        1.0     into   ADP          O
9        1.0      the   DET          O

<IPython.core.display.Javascript object>

In [8]:
data["tag"].value_counts()

O               2498
U-Action         366
U-Ingredient     241
L-Ingredient      70
B-Ingredient      70
B-Utensil         60
L-Utensil         59
U-Utensil         59
I-Utensil         15
I-Ingredient       3
L-Action           2
B-Action           2
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [41]:
print(len(data["word"].values))

3445


<IPython.core.display.Javascript object>

In [9]:
words = list(set(data["word"].values))
len(words)

869

<IPython.core.display.Javascript object>

In [10]:
agg_func = lambda s: [
    (w, p, t)
    for w, p, t in zip(
        s["word"].values.tolist(), s["pos"].values.tolist(), s["tag"].values.tolist()
    )
]

<IPython.core.display.Javascript object>

In [11]:
grouped = data.groupby("sentence#").apply(agg_func)

<IPython.core.display.Javascript object>

In [12]:
sentences = [s for s in grouped]

<IPython.core.display.Javascript object>

In [13]:
len(sentences)

264

<IPython.core.display.Javascript object>

In [14]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        # set the bias for each word
        "bias": 1.0,
        # get the lower case form of the word
        "word.lower()": word.lower(),
        # get last 3 letters for the word
        "word[-3:]": word[-3:],
        # get last 2 letters for the word
        "word[-2:]": word[-2:],
        # check whether the word is uppercase or not
        "word.isupper()": word.isupper(),
        # check whether the word is title case or not
        "word.istitle()": word.istitle(),
        # check whether the word is digit or not, useful to identifying quantities which will be tagged as 'O'
        "word.isdigit()": word.isdigit(),
        # specifying the pos for word
        "postag": postag,
        # get first 2 letters for the POS tag
        "postag[:2]": postag[:2],
    }

    # if word is starting of sentence
    if i > 0:

        # if word is not the beginning of sentence
        # then get the word before it i.e. i-1 index
        word1 = sent[i - 1][0]

        # then get the pos before it i.e. i-1 index
        postag1 = sent[i - 1][1]

        features.update(
            {
                # setting the lower form of word at index i-1
                "-1:word.lower()": word1.lower(),
                # checking if the word at index i-1 is titlecase
                "-1:word.istitle()": word1.istitle(),
                # checking if the word at index i-1 is uppercase
                "-1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i-1
                "-1:postag": postag1,
                # get first 2 letters for the POS tag for i-1 indexed word
                "-1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the BOS or Begining of sentence to True
        features["BOS"] = True

    # if word is at the end of sentence
    if i < len(sent) - 1:

        # if word is not the end of sentence
        # then get the word after it i.e. i+1 index
        word1 = sent[i + 1][0]

        # then get the pos after it i.e. i+1 index
        postag1 = sent[i + 1][1]

        features.update(
            {
                # setting the lower form of word at index i+1
                "+1:word.lower()": word1.lower(),
                # checking if the word at index i+1 is titlecase
                "+1:word.istitle()": word1.istitle(),
                # checking if the word at index i+1 is titlecase
                "+1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i+1
                "+1:postag": postag1,
                # get first 2 letters for the POS tag for i+1 indexed word
                "+1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the EOS or End of sentence to True
        features["EOS"] = True

    return features

<IPython.core.display.Javascript object>

In [15]:
def sent2features(sent):
    """Convert sentences which are lists containing (w, p, t) into features"""
    return [word2features(sent, i) for i in range(len(sent))]

<IPython.core.display.Javascript object>

In [16]:
def sent2labels(sent):
    """Retrieve all the labels from sentences which are lists containing (w, p, t)"""
    return [label for token, postag, label in sent]

<IPython.core.display.Javascript object>

In [17]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

<IPython.core.display.Javascript object>

In [18]:
# split into train and test
boundary = math.ceil(len(X) * 0.8)

<IPython.core.display.Javascript object>

In [19]:
boundary

212

<IPython.core.display.Javascript object>

In [20]:
# train data
x_train = X[:boundary]
y_train = y[:boundary]

# test data
x_test = X[boundary:]
y_test = y[boundary:]

<IPython.core.display.Javascript object>

In [21]:
print(len(x_train))
print(len(x_test))

212
52


<IPython.core.display.Javascript object>

In [22]:
# creating CRF model with Gradient Descent
crf = CRF(
    algorithm="lbfgs",
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
)

<IPython.core.display.Javascript object>

In [23]:
# fitting the model using train data
crf.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<IPython.core.display.Javascript object>

In [24]:
# to get all the labels/tags of data
labels = list(crf.classes_)

<IPython.core.display.Javascript object>

In [25]:
labels

['U-Action',
 'U-Utensil',
 'O',
 'B-Utensil',
 'L-Utensil',
 'U-Ingredient',
 'B-Ingredient',
 'L-Ingredient',
 'I-Utensil',
 'I-Ingredient',
 'B-Action',
 'L-Action']

<IPython.core.display.Javascript object>

Since we are not interested in 'O' tags we will check the performance of the CRF model using f1 scores for every tag except O tags.

In [26]:
labels.remove("O")
labels

['U-Action',
 'U-Utensil',
 'B-Utensil',
 'L-Utensil',
 'U-Ingredient',
 'B-Ingredient',
 'L-Ingredient',
 'I-Utensil',
 'I-Ingredient',
 'B-Action',
 'L-Action']

<IPython.core.display.Javascript object>

In [27]:
# performing predictions based on the fitted model
y_pred = crf.predict(x_test)

<IPython.core.display.Javascript object>

In [28]:
# finding the f1 score
metrics.flat_f1_score(y_test, y_pred, average="weighted", labels=labels)

0.8541805862275337

<IPython.core.display.Javascript object>

In [29]:
# finding the f1 score
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

              precision    recall  f1-score   support

    U-Action      0.964     0.976     0.970        82
   U-Utensil      1.000     0.529     0.692        17
   B-Utensil      0.778     0.538     0.636        13
   L-Utensil      0.778     0.538     0.636        13
U-Ingredient      0.860     0.956     0.905        45
B-Ingredient      0.647     0.917     0.759        12
L-Ingredient      0.647     0.917     0.759        12
   I-Utensil      1.000     0.333     0.500         3
I-Ingredient      0.000     0.000     0.000         0
    B-Action      0.000     0.000     0.000         0
    L-Action      0.000     0.000     0.000         0

   micro avg      0.867     0.858     0.862       197
   macro avg      0.607     0.519     0.532       197
weighted avg      0.881     0.858     0.854       197



<IPython.core.display.Javascript object>

The model overfits!!

In [47]:
len(data[((data["word"] == "Boil") | (data["word"] == "boil"))])

13

<IPython.core.display.Javascript object>

In [48]:
len(data[((data["word"] == "Cover") | (data["word"] == "cover"))])

17

<IPython.core.display.Javascript object>

In [49]:
len(data[((data["word"] == "Preheat") | (data["word"] == "preheat"))])

9

<IPython.core.display.Javascript object>

In [40]:
eli5.show_weights(crf, top=30)

<IPython.core.display.Javascript object>

**Hyperparameter Optimization**

In [31]:
# creating a CRF Hyperparameter tuned model
crf_hp = CRF(
    algorithm="lbfgs", c1=10, c2=0.1, max_iterations=100, all_possible_transitions=False
)

# parameters to hypertune
params_space = {"c1": scipy.stats.expon(scale=0.5), "c2": scipy.stats.expon(scale=0.05)}

# metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average="weighted", labels=labels)

# search using 3 fold cross validation
rs = RandomizedSearchCV(
    crf_hp, params_space, cv=3, verbose=1, n_jobs=-1, n_iter=50, scoring=f1_scorer
)

<IPython.core.display.Javascript object>

In [32]:
# fitting the hyperparameters
rs.fit(x_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=False, averaging=None,
                                 c=None, c1=10, c2=0.1,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_te...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000013ECA0E6E48>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighte

<IPython.core.display.Javascript object>

Best Parameters:

In [33]:
print("best params:", rs.best_params_)
print("best CV score:", rs.best_score_)

best params: {'c1': 0.0027687101879677375, 'c2': 0.08449483887430631}
best CV score: 0.8782080343880696


<IPython.core.display.Javascript object>

In [34]:
# set the best estimator
crf_hp_tuned = rs.best_estimator_

<IPython.core.display.Javascript object>

In [35]:
# predict using the best CRF model
y_pred = crf_hp_tuned.predict(x_test)

<IPython.core.display.Javascript object>

In [36]:
# print the f1 evaluation metric
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

              precision    recall  f1-score   support

    U-Action      0.952     0.976     0.964        82
   U-Utensil      1.000     0.529     0.692        17
   B-Utensil      0.778     0.538     0.636        13
   L-Utensil      0.778     0.538     0.636        13
U-Ingredient      0.854     0.911     0.882        45
B-Ingredient      0.647     0.917     0.759        12
L-Ingredient      0.647     0.917     0.759        12
   I-Utensil      1.000     0.333     0.500         3
I-Ingredient      0.000     0.000     0.000         0
    B-Action      0.000     0.000     0.000         0
    L-Action      0.000     0.000     0.000         0

   micro avg      0.861     0.848     0.854       197
   macro avg      0.605     0.515     0.530       197
weighted avg      0.875     0.848     0.846       197



<IPython.core.display.Javascript object>

In [37]:
# transition features: given B-Utensil then what is score for I-Utensil

<IPython.core.display.Javascript object>

In [38]:
# state features: what property of feature describes the tag

<IPython.core.display.Javascript object>

In [39]:
eli5.show_weights(crf_hp_tuned, top=30)

<IPython.core.display.Javascript object>

In [50]:
spacy.explain("JJ")

NameError: name 'spacy' is not defined

<IPython.core.display.Javascript object>